In [1]:
import numpy as np # 
import pandas as pd # 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/Users/zennureozbekacar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


In [2]:
# load data
df = pd.read_json('resources/News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,She left her husband. He killed their children. Just another day in America.,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,The actor and his longtime girlfriend Anna Eberstein tied the knot in a civil ceremony.,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,The actor gives Dems an ass-kicking for not fighting hard enough against Donald Trump.,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,"The ""Dietland"" actress said using the bags is a ""really cathartic, therapeutic moment.""",2018-05-26


In [3]:
df = df.drop_duplicates(subset='short_description', keep='last')

In [4]:
df=df.reset_index()

In [5]:
import re
import string

def clean_text_round(text):
    
    text = text.lower()
    text = re.sub('\[.*?\]', '',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[''""...]','',text)
    text = re.sub('\n','',text)
    text = re.sub('\d+', '', text)
    text = re.sub('[^\w\s]','', text)  # remove punctuation 
    
    return text

get_cleaning = lambda x: clean_text_round(x)
df.short_description = pd.DataFrame(df.short_description.apply(get_cleaning))
df.short_description = df.short_description.str.replace('\d+', '') # remove numbers

In [46]:
import gensim
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import remove_stopwords

In [47]:
df['short_description'] = df['short_description'].apply(lambda x: gensim.parsing.preprocessing.remove_stopwords(x))



In [48]:
df.short_description[2]

'actor longtime girlfriend anna eberstein tied knot civil ceremony'

In [49]:
remove_words = ['i', 'your', 'you', 'on', 'with', 'and', 'have', 'the', 'to', 'in', 'for', 'that', 'had', 'be', 'a', 'year'
               'it', 'may', 'one', 'as', 'if', 'is', 'via', 'this', 'will', 'david', 'jenni', 'im', 'susan', 'it', 'up',
               'angelica', 'hi', 'hello', 'we', 'our', 'all', 'kelli', 'yes', 'gavin', 'our', 'were', 'of', 'can', 'at',
               'any', 'by', 'also', 'joe', 'ronnie', 'morning', 'evening', 'good', 'what', 'okay', 'ok', 'are', 'us', 'my',
               'th', 'st', 'nd', 'rd', 'was', 'there', 'then', 'lee', 'out', 'or', 'so','alan', 'from','unfortunately',
               'alason', 'but', 'youre', 'does', 'heres', 'little', 'more', 'set', 'br', 'dr', 'ave', 'here', 'about', 'an',
               'let', 'know', 'than', 'then', 'no', 'why', 'way', 'every', 'thats', 'taken', 'today', 'way', 'id', 'isnt', 
               'only', 'bay', 'me', 'when', 'want', 'end', 'month', 'do', 'til', 'get', 'back', 'thanks', 'bonnie', 'woodal',
               'off', 'drive', 'michell','and', 'he','she', 'her','just']

In [50]:
def apply_remove_words_to_texts():
    pat = r'\b(?:{})\b'.format('|'.join(remove_words))
    df["short_description_new"] = df['short_description'].str.replace(pat, '')
    df["short_description_new"] = df['short_description_new'].str.strip()
    return df

In [51]:
df = apply_remove_words_to_texts()

In [52]:
df.head()

,index,category,headline,authors,link,short_description,date,short_description_new,label
0,0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,left husband killed children day america,2018-05-26,left husband killed children day america,Negative
1,1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,course song,2018-05-26,course song,Neutral
2,2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,actor longtime girlfriend anna eberstein tied knot civil ceremony,2018-05-26,actor longtime girlfriend anna eberstein tied knot civil ceremony,Neutral
3,3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,actor gives dems asskicking fighting hard donald trump,2018-05-26,actor gives dems asskicking fighting hard donald trump,Positive
4,4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,dietland actress said bags cathartic therapeutic moment,2018-05-26,dietland actress said bags cathartic therapeutic moment,Neutral


In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [54]:
analyzer = SentimentIntensityAnalyzer()
overall_sentiment = []

for art in df['short_description_new']:
    sentiment_dict=analyzer.polarity_scores(art)
    #print("Overall sentiment dictionary is : ", sentiment_dict) 
    #print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative") 
    #print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral") 
    #print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive") 
    #print("Sentence Overall Rated As", end = " ")  
    if sentiment_dict['compound'] >= 0.05 : 
        overall_sentiment.append("Positive")
    elif sentiment_dict['compound'] <= - 0.05 : 
        overall_sentiment.append("Negative")
    else : 
        overall_sentiment.append("Neutral")
        
df['label'] = overall_sentiment

In [55]:
df.head()

,index,category,headline,authors,link,short_description,date,short_description_new,label
0,0,CRIME,"There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV",Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-amanda-painter-mass-shooting_us_5b081ab4e4b0802d69caad89,left husband killed children day america,2018-05-26,left husband killed children day america,Negative
1,1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song,Andy McDonald,https://www.huffingtonpost.com/entry/will-smith-joins-diplo-and-nicky-jam-for-the-official-2018-world-cup-song_us_5b09726fe4b0fdb2aa541201,course song,2018-05-26,course song,Neutral
2,2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-grant-marries_us_5b09212ce4b0568a880b9a8c,actor longtime girlfriend anna eberstein tied knot civil ceremony,2018-05-26,actor longtime girlfriend anna eberstein tied knot civil ceremony,Neutral
3,3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carrey-adam-schiff-democrats_us_5b0950e8e4b0fdb2aa53e675,actor gives dems asskicking fighting hard donald trump,2018-05-26,actor gives dems asskicking fighting hard donald trump,Negative
4,4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-margulies-trump-poop-bag_us_5b093ec2e4b0fdb2aa53df70,dietland actress said bags cathartic therapeutic moment,2018-05-26,dietland actress said bags cathartic therapeutic moment,Neutral


In [56]:
df['category'].value_counts()

POLITICS          29106
WELLNESS          17549
ENTERTAINMENT     13251
TRAVEL            9375 
STYLE & BEAUTY    9152 
PARENTING         8624 
FOOD & DRINK      6206 
QUEER VOICES      5405 
HEALTHY LIVING    5197 
BUSINESS          5060 
COMEDY            4361 
SPORTS            4186 
BLACK VOICES      4064 
HOME & LIVING     4023 
THE WORLDPOST     3657 
WEDDINGS          3645 
PARENTS           3433 
DIVORCE           3401 
IMPACT            3050 
WOMEN             3023 
CRIME             2671 
MEDIA             2259 
WORLD NEWS        2169 
WEIRD NEWS        2152 
GREEN             2034 
TECH              1990 
TASTE             1904 
RELIGION          1779 
SCIENCE           1763 
MONEY             1706 
STYLE             1500 
ARTS & CULTURE    1321 
ENVIRONMENT       1317 
WORLDPOST         1242 
FIFTY             1042 
GOOD NEWS         1027 
LATINO VOICES     1018 
CULTURE & ARTS    1016 
COLLEGE           921  
EDUCATION         891  
ARTS              863  
Name: category, 

News API categories
* business
* entertainment
* general
* health
* science
* sports
* technology

In [57]:
business =['BUSINESS','MONEY']
entertainment=['ENTERTAINMENT','ARTS','COMEDY','ARTS & CULTURE','CULTURE & ARTS','MEDIA']
health=['HEALTH','WELLNESS','HEALTHY LIVING']
science=['SCIENCE']
sports=['SPORTS']
technology = ['tech']

In [58]:
condition = df['category'].isin(business)
businessDF = df[condition]
businessDF.shape

(6766, 9)

In [59]:
from sklearn.model_selection import train_test_split

trainData, testData = train_test_split(businessDF, test_size=0.3)

In [60]:
len(businessDF)

6766

In [61]:
len(trainData)

4736

In [62]:
len(testData)

2030

In [63]:
trainData.head()

,index,category,headline,authors,link,short_description,date,short_description_new,label
177645,200139,BUSINESS,Mortgage Deal Reached In 2008 Shows Pitfalls To Avoid In Current Settlement,Loren Berlin,https://www.huffingtonpost.com/entry/mortgage-deal-settlement-banks-attorney-general_us_5bb2b49ae4b0480ca659e69f,obama administration pushing states sign new settlement agrees countrywide deal underwhelmed,2012-02-05,obama administration pushing states sign new settlement agrees countrywide deal underwhelmed,Positive
167976,190290,MONEY,Why We All Need to Think About Alternative Credit,"Credit.com, Contributor\nExpert information about credit scoring, credit reporting and ...",https://www.huffingtonpost.com/entry/why-we-all-need-to-think_us_5b9bbd22e4b03a1dcc79f834,time alternative credit join mainstream credit conversation long overdue youre consumer,2012-05-21,time alternative credit join mainstream credit conversation long overdue consumer,Positive
32377,32879,BUSINESS,Bay Area School Gets Rich Quick On Snapchat Investment,Ron Dicker,https://www.huffingtonpost.com/entry/st-francis-high-snapchat-investment_us_58b95626e4b0d2821b4d411f,gods looking president catholic high school said,2017-03-03,gods looking president catholic high school said,Neutral
136626,158396,BUSINESS,This FAA Sequester Vote Doesn't Smell Right,"Jared Bernstein, Contributor\nFmr. Obama administration economist; CNBC and MSNBC contributor",https://www.huffingtonpost.com/entry/faa-furloughs-sequestration_us_5bb2f2c1e4b0480ca65f4ab9,bipartisan support mitigate noxious effects sequestration papering highvisibility stuff affects affluent lots budget bleeding goes beneath radar,2013-04-27,bipartisan support mitigate noxious effects sequestration papering highvisibility stuff affects affluent lots budget bleeding goes beneath radar,Positive
54043,55733,BUSINESS,Judge Orders Leonardo DiCaprio To Give Deposition In 'Wolf Of Wall Street' Lawsuit,,https://www.huffingtonpost.com/entry/leonardo-dicaprio-deposition-wolf-wall_us_57648c68e4b015db1bc98231,plaintiff sued million claiming defamed film,2016-06-18,plaintiff sued million claiming defamed film,Neutral


In [64]:
trainData.to_csv("Resources/BusinessTrainData_1.csv")
testData.to_csv("Resources/BusinessTestData_1.csv")

In [65]:
trainData.sample(frac=1).head(5) # shuffle the df and pick first 5

,index,category,headline,authors,link,short_description,date,short_description_new,label
126245,147817,BUSINESS,States Profiting The Most From Sin: 24/7 Wall St.,"24/7 Wall St., 24/7 Wall St.",https://www.huffingtonpost.com/entry/sin-states_us_5bb2fbf9e4b0480ca6605e93,like tax foundations scott drenkard argue states motivated profit specifically increasing,2013-08-17,like tax foundations scott drenkard argue states motivated profit specifically increasing,Positive
17388,17631,BUSINESS,Hurricane Harvey And The Failure Of The Free Market,"Paul Constant, ContributorFellow at Civic Ventures and co-founder of the Seattle Review ...",https://www.huffingtonpost.com/entry/hurricane-harvey-and-the-failure-of-the-free-market_us_59a6ce80e4b0d81379a81c8e,lifeordeath situation,2017-08-30,lifeordeath situation,Neutral
142441,164325,BUSINESS,Happy Birthday Steve Jobs: Business Leaders Reflect On Ex-Apple CEO,,https://www.huffingtonpost.com/entry/happy-birthday-steve-jobs_us_5bb2e9abe4b0480ca65e9725,exapple ceo steve jobs alive today following death cancer age major,2013-02-24,exapple ceo steve jobs alive following death cancer age major,Negative
152740,174777,BUSINESS,Presidential Election 2012: Let the Best Brand Win,"Mario Simon, Contributor\nCEO, Millward Brown Vermeer",https://www.huffingtonpost.com/entry/candidate-branding_us_5bb2da4ee4b0480ca65d5344,political leaders known centuries commercial organizations recently discovered successful brands rooted universal human ideals,2012-11-04,political leaders known centuries commercial organizations recently discovered successful brands rooted universal human ideals,Positive
67676,70009,BUSINESS,How To Ask For A Raise,,https://www.huffingtonpost.comhttp://pubx.co/nPghPj,youve summoned ask raise youre definitely,2016-01-07,youve summoned ask raise definitely,Positive


# Vectorizing the Data

In [66]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['short_description_new'])
test_vectors = vectorizer.transform(testData['short_description_new'])

In [67]:
train_vectors

<4736x2497 sparse matrix of type '<class 'numpy.float64'>'
	with 41195 stored elements in Compressed Sparse Row format>

In [68]:
test_vectors

<2030x2497 sparse matrix of type '<class 'numpy.float64'>'
	with 16808 stored elements in Compressed Sparse Row format>

# Creating a Linear SVM Mode

In [69]:
import time
# from sklearn import svm
from sklearn.svm import SVC 
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn import set_config

In [70]:
# Perform classification with SVM, kernel=linear
classifier = SVC(kernel='linear')
t0 = time.time()
t1 = time.time()
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
set_config(display="diagram")
pipe = Pipeline([('classifier', classifier)])
pipe

Pipeline(steps=[('classifier', SVC(kernel='linear'))])

In [71]:
# classifier.fit(train_vectors, trainData['label'])
pipe.fit(train_vectors, trainData['label'])

Pipeline(steps=[('classifier', SVC(kernel='linear'))])

In [72]:
predicted=pipe.predict(test_vectors)

In [73]:
svm_acc_train=pipe.score(train_vectors, trainData['label'])
svm_acc_test=pipe.score(test_vectors, testData['label'])

In [77]:
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['label'], predicted, output_dict=True)
print(f"Train Accuracy = {svm_acc_train}")
print(f"Test Accuracy = {svm_acc_test}")
print('positive: ', report['Positive'])
print('negative: ', report['Negative'])

Training time: 0.000031s; Prediction time: 0.000026s
Train Accuracy = 0.8754222972972973
Test Accuracy = 0.6610837438423646
positive:  {'precision': 0.7033824804856895, 'recall': 0.8343621399176955, 'f1-score': 0.7632941176470588, 'support': 972}
negative:  {'precision': 0.6541019955654102, 'recall': 0.5221238938053098, 'f1-score': 0.5807086614173229, 'support': 565}


In [75]:
from sklearn.metrics import classification_report
print(classification_report(pipe.predict(test_vectors),testData['label']))

              precision    recall  f1-score   support

    Negative       0.52      0.65      0.58       451
     Neutral       0.48      0.55      0.51       426
    Positive       0.83      0.70      0.76      1153

    accuracy                           0.66      2030
   macro avg       0.61      0.64      0.62      2030
weighted avg       0.69      0.66      0.67      2030



# Save the Model

In [ ]:
import pickle
# pickling the vectorizer
pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
# pickling the model
pickle.dump(classifier_linear, open('classifier.sav', 'wb'))

# Test the SVM classifier

In [ ]:
review = """SUPERB, I AM IN LOVE IN THIS PHONE"""
review_vector = vectorizer.transform([review]) # vectorizing
print(classifier_linear.predict(review_vector))

In [78]:
# f = open('Resources/BusinessTrainData_1.csv', 'r', encoding='utf8')
# text = f.read()
# t_list = text.split('\n')

# corpus = []

# for sentence in t_list:
#     corpus.append(sentence.split())


In [79]:
# print(corpus[:10])

In [80]:
# vector = Word2Vec(corpus, size=100, min_count=1,sg=1)

In [81]:
# vector.wv['app', 'council']

In [82]:
# vector.wv.most_similar('council')

In [83]:
# vector.wv.most_similar('app')

In [84]:
# vector.save('word2vec.vector')

In [85]:
# vector = Word2Vec.load('word2vec.vector')

In [86]:
# def closestwords_tsneplot(model, word):
#     word_vectors = np.empty((0,100))
#     word_labels = [word]
    
#     close_words = model.wv.most_similar(word)
    
#     word_vectors = np.append(word_vectors, np.array([model.wv[word]]), axis=0)
    
#     for w, _ in close_words:
#         word_labels.append(w)
#         word_vectors = np.append(word_vectors, np.array([model.wv[w]]), axis=0)
        
#     tsne = TSNE(random_state=0)
#     Y = tsne.fit_transform(word_vectors)
    
#     x_coords = Y[:, 0]
#     y_coords = Y[:, 1]
    
#     plt.scatter(x_coords, y_coords)
    
#     for label, x, y in zip(word_labels, x_coords, y_coords):
#         plt.annotate(label, xy=(x, y), xytext=(5, -2), textcoords='offset points')
        
#     plt.show()

In [87]:
# closestwords_tsneplot(model, 'council')

In [88]:
# closestwords_tsneplot(model, 'firearms')

In [89]:
# closestwords_tsneplot(model, 'Bezos')

In [90]:
# closestwords_tsneplot(model, 'scandal')